In [10]:
import pandas as pd
import numpy as np
from pywt import wavedec

def wavelet_avg_features(data, type_wav, epsilon=1e-10):
    coeffs = wavedec(data, type_wav, level=5)
    
    def safe_log(x, eps=epsilon):
        return np.log(x + eps)
    
    cD_Energy = np.mean([np.sum(np.square(coeffs[i])) for i in range(1, 6)])
    cA_Energy = np.sum(np.square(coeffs[0]))
    
    D_Entropy = np.mean([np.sum(np.square(coeffs[i]) * safe_log(np.square(coeffs[i]))) for i in range(1, 6)])
    A_Entropy = np.sum(np.square(coeffs[0]) * safe_log(np.square(coeffs[0])))
    
    D_mean = np.mean([np.mean(coeffs[i]) for i in range(1, 6)])
    A_mean = np.mean(coeffs[0])
    
    D_std = np.mean([np.std(coeffs[i]) for i in range(1, 6)])
    A_std = np.std(coeffs[0])
    
    features = [cD_Energy, cA_Energy, D_Entropy, A_Entropy, D_mean, A_mean, D_std, A_std]
    return features

def save_to_csv(df, filename, normalize=True):
    if normalize:
        df = (df - df.mean()) / df.std()
    df.to_csv(filename + '.csv', index=False)

# Cargar datos desde un archivo de Excel usando openpyxl
data = pd.read_excel('Nadira.xlsx', header=None, engine='openpyxl')

# Suponiendo que la data está dividida en 8 filas, cada fila representa un canal
channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4']

window_size = 40  # Tamaño de ventana para dividir los datos

df = pd.DataFrame()
for i, ch in enumerate(channels):
    chanel_data = data.iloc[i, :].dropna().values  # Extraer los datos de cada canal y eliminar NaNs
    features_list = []
    num_windows = len(chanel_data) // window_size
    
    print(f"Processing channel {ch} with {num_windows} windows")  # Mensaje de depuración
    
    for w in range(num_windows):
        row = chanel_data[w * window_size:(w + 1) * window_size]
        features = wavelet_avg_features(row, 'db4')
        features_list.append(features)
    
    if not features_list:
        print(f"No features calculated for channel {ch}")  # Mensaje de depuración

    df_channel = pd.DataFrame(features_list, columns=[f'{ch}_cD_Energy', f'{ch}_cA_Energy', f'{ch}_D_Entropy', f'{ch}_A_Entropy', f'{ch}_D_mean', f'{ch}_A_mean', f'{ch}_D_std', f'{ch}_A_std'])
    df = pd.concat([df, df_channel], axis=1)

save_to_csv(df, 'wave_all_channel_8_Feature')

print("Data processing complete. CSV file saved.")


Processing channel Fp1 with 156 windows
Processing channel Fp2 with 156 windows
Processing channel F3 with 156 windows
Processing channel F4 with 156 windows
Processing channel C3 with 156 windows
Processing channel C4 with 156 windows
Processing channel P3 with 156 windows
Processing channel P4 with 156 windows
Data processing complete. CSV file saved.
